# Homework 2: Nannon
### By Ari Porad
### For COSI 101A with Professor Jordan Pollack, April 12th 2021

## Discussion

For this assignment, I built an implementation of the game Nannon and several different algorithms that could play it. Some high-level discussion of my approach follows in this section, then I'll walk through my implementation in more detail.

### Representation

I'm a big believer that if you choose your data structures right, the rest of your program writes itself. (That's a rough quote from someone, but I can't remember who.) To that end, I spent a substantial amount of time during this project iterating on my data structures. Originally, I stored the board as a list, where each item in the list represented a position on the board. Legal values were `0` (spot is empty), `1` (current player's checker), and `-1` (opponent's checker). Checkers in the home and goal zones were stored separately. That data structure was simple, which was nice. However, it was easy to get into an illegal state (wrong number of checkers), which I didn't like--a good data structure is one that can't represent an illegal state, after all. It was also hard to manipulate, with lots of loops looking for checkers. I ultimately decided to switch to a representation with an (unordered) list of `Checker`s, each of which has a `Player` (black or white) and a `Position`. Initially, I attempted to have each checker's position be relative _to that checker's home_, which meant that the representation was entirely perspective-independent. That ended up being too complicated to manage, so I settled on a perspective-dependent representation where all checker positions are measured relative to one player's (the perspective player's) home. I'm still not satisfied with this representation--it simultaneously feels a little too complicated, while also needing too much perspective swapping--but it's good enough.

The list of checkers (which is always `2 * checkers_per_player` long) is encapsulated by a `Board`, which also tracks the perspective (if the perspective is Black, then lower indexes are closer to Black's home and vis versa) and `GameConfiguration` (the variant of Nannon, such as `{6,3,6}` or `{8,4,6}`). `Board` contains much of the game logic, including calculating open spots, legal `Move`s, and the winner. The `Move` class trackes a possible move, and is responsible for properly executing it and returning the resulting `Board`. Finally, a `Dicestream` class wraps various iterators that can provide a stream of dice rolls.

### Knowledge-Based Player

My knowledge-based Nannon player ended up somewhat similar to the score-based player, but with some key differences. It's built by assigning "points" to different aspects of each possible move. It picks the move with the highest total point value.

The algorithm's guiding principles are:

* Checkers closer to the goal are better
* Knocking off an opponent's checker is valuable, especially if it's close to the goal
* Using a roll of 6 to move a checker that's 1 away from the goal is a waste

There were a couple of things that I expected to result in significant improvements, but in testing either had negligible or negative impacts, so aren't part of the final algorithm:
* Favoring protected checkers
* Trying to account for the risk of getting trapped
* Valuing checkers closer to the goal non-linearly (ie. a checker right next to the goal is _way_ more valuable than one right next to home)
* Optimizing for the number of legal moves

### Sticking Points

As mentioned above, picking (and iterating on) a set of datastructures was a big sticking point for this assignment. Building a knowledge-based player was also more difficult than anticipated--the strategies that I intuitively expected to work well weren't actually that effective, whereas the very simple score-based player was actually quite good (as, was somewhat surprisingly, the simple last-piece-first player). It's also quite possible that I'm just bad at Nannon--as any of my friends or family can attest, I'm not generally not one for playing games like these the old-fashion way. Additionally, I ran into some very very strange situations that I think are bugs in the Python interpreter, which are more thoroughly documented in `cli.py`--but basically, `Player.BLACK.long_str` would sometimes return `'White'`, but only in the PyCharm debugger and inconsistently. That was a fun one.

### Performance

I spent a substantial amount of time optimizing the performance of this solution. Running a profiler against a tournament led me to introduce significant memoization (using Python's [`functools.cache`/`functools.cached_property`][caching]) throught the system, and to optimize the equality checking and hashing of various classes. This ultimately resulted in a 2-3x performance increase over the initial version.

[caching]: https://docs.python.org/3/library/functools.html

## Running the Code

I hope I've provided enough information in this report that--as requested--you won't need to run the code. If you do, however, there are a couple of ways to do so.

First, much of the code, especially the game foundation, is unit tested through Python's [doctest][] module. Tests can be run from the command line:

```bash
# This prints nothing if the tests pass
$ python3 -m doctest src/*.py
```

Additionally, I built a simple CLI for interacting with the game system:

```bash
$ python3 src/cli.py --help
usage: cli.py [-h] [-n ROUNDS] [-b BOARD_SIZE] [-c CHECKERS] [-d DIE_SIZE] [-s SEED]
              {g,game,t,tournament,r,roundrobin} {first,human,knowledge,last,random,score} [{first,human,knowledge,last,random,score} ...]

positional arguments:
  {g,game,t,tournament,r,roundrobin}
                        game mode
  {first,human,knowledge,last,random,score}
                        player algorithms (2 for game our tournament, 2+ for round robin)

optional arguments:
  -h, --help            show this help message and exit
  -n ROUNDS, --rounds ROUNDS
                        number of matches to run
  -b BOARD_SIZE, --board-size BOARD_SIZE
                        board size
  -c CHECKERS, --checkers CHECKERS
                        checkers per player
  -d DIE_SIZE, --die-size DIE_SIZE
                        die size
  -s SEED, --seed SEED  the seed for the random roll generator

$ python3 src/cli.py game random human  # play against a random player
<Output not shown because it requires humna input>

$ python3 src/cli.py tournament random first last score -n 3000  # run a tournament and show the aggregate results
Playing a Nannon{6,3,6} bulk tournament of 3000 games each between random, first, last, score. seed = 1618284126
Played 6 tournaments of 3000 games each. Took 3s.
Loser → | first   | last    | random  | score
-----------------------------------------------
random  | 53.5%   | 45.8%   | -       | 46.8%
first   | -       | 44.9%   | 46.5%   | 43.0%
last    | 55.1%   | -       | 54.2%   | 50.2%
score   | 57.0%   | 49.8%   | 53.2%   | -

$ python3 src/cli.py tournament last score first --seed 12345  # set the seed to any integer for consistent dice (doesn't affect the random player's decisions). We'll use the same seed as above
Playing a Nannon{6,3,6} bulk tournament of 100 games each between last, score, first. seed = 12345
Played 3 tournaments of 100 games each. Took 0s.
Loser → | first   | last    | score
-------------------------------------
last    | 55.0%   | -       | 48.0%
score   | 52.0%   | 52.0%   | -
first   | -       | 45.0%   | 48.0%
```

[doctest]: https://docs.python.org/3/library/doctest.html

## Results

First off, let's run a round-robin tournament and look at the results. We'll run a large number of matches to get more consistent results.

In [1]:
from bulk_tournament import bulk_tournament
from structs import *
from Dicestream import *
from Board import *
from players import *

results = bulk_tournament([
    LastPlayerAlgorithm(),
    FirstPlayerAlgorithm(),
    ScorePlayerAlgorithm(),
    RandomPlayerAlgorithm(),
    KnowledgePlayerAlgorithm()
], rounds=10000, config=GameConfiguration(6, 3, 6), seed=1618293516)

Playing a Nannon{6,3,6} bulk tournament of 10000 games each between last, first, score, random, knowledge. seed = 1618293516
Played 10 tournaments of 10000 games each. Took 21s.                                                                              
Loser →   | first     | knowledge | last      | random    | score    
---------------------------------------------------------------------
last      | 62.0%     | 42.6%     | -         | 59.7%     | 48.2%    
first     | -         | 30.0%     | 38.0%     | 46.8%     | 35.8%    
score     | 64.2%     | 43.9%     | 51.8%     | 61.0%     | -        
random    | 53.2%     | 33.3%     | 40.3%     | -         | 39.0%    
knowledge | 70.0%     | -         | 57.4%     | 66.7%     | 56.1%    


That ran each a 10,000 game match between each of 5 players (for total of 100,000 matches), showing that (for example), the `last` algorithm triumphs over the `first` algorithm 62% of the time.

## Walthrough

Now, I'll walk you through each part of the assignment:

### Part 1: Basic Functions and Game Logic

In [2]:
# Dicestreams
rand = Dicestream.random(die_size=6, seed=1)
print("Random Roll:", rand.roll())
print("Random Roll:", rand.roll())
print("Random Roll:", rand.roll())
dicestream = Dicestream.fixed([1, 5, 3])
print("This will be 1:", dicestream.roll())
print("This will be 2 (p1 rolls 5, p2 rolls 3):", dicestream.first_roll())

Random Roll: 2
Random Roll: 5
Random Roll: 1
This will be 1: 1
This will be 2 (p1 rolls 5, p2 rolls 3): 2


In [3]:
# Board Representation and Swapping
print("Here's a starting board from Black's perspective:")
print(Board.create_starting_board(perspective=Player.BLACK).draw())
print("Here's the same board, but swapped:")
print(Board.create_starting_board(perspective=Player.BLACK).swapped.draw())
print("Here's a board where Black won:")
black_won = Board(perspective=Player.BLACK, board=('', '-----W', 'BBBWW'))
print("Who Won? ", black_won.whowon)  # Black
print(black_won.draw())

Here's a starting board from Black's perspective:
           ● Black →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ●  ●        ○  ○  |
Home → ●   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ○ ← Home
                        ← White ○
Here's the same board, but swapped:
           ○ White →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ○  ○        ●  ●  |
Home → ○   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ● ← Home
                        ← Black ●
Here's a board where Black won:
Who Won?  Player.BLACK
           ● Black →
Goal → ○○  |  ▼  ▼  ▼  ▼  ▼  ▼  | ●●● ← Goal
           |                 ○  |
Home →     |  ▲  ▲  ▲  ▲  ▲  ▲  |     ← Home
                        ← White ○


In [4]:
# Legal Move Generation (hyphen represents the checker's previous location)
print("Here's a board where white will move from spot 2 to spot 5 (zero-indexed):")
move = Board(perspective=Player.WHITE, board=('', 'B-W---', 'BBWW')) \
    .legal_moves(3, player=Player.WHITE)[0]
print(move)
print(move.draw())

Here's a board where white will move from spot 2 to spot 5 (zero-indexed):
Move(W: 2 -> 5)
           ○ White →
Goal → ●●  |  ▼  ▼  ▼  ▼  ▼  ▼  |  ○○ ← Goal
           |  ●     -        ○  |
Home →     |  ▲  ▲  ▲  ▲  ▲  ▲  |     ← Home
                        ← Black ●


### Part 2: Generalize the Game

How many possible boards are there in various permutations of Nannon?

In [5]:
from explore import explore

# explore() takes a while to run, so I've hard-coded the outputs (from a previous run) below for convenience
num_boards_636 = 2529
num_boards_846 = 31102
num_boards_1056 = 367241
recalculate = False

if recalculate:
    num_boards_636 = len(explore(GameConfiguration(6, 3, 6)))
    num_boards_846 = len(explore(GameConfiguration(8, 4, 6)))
    num_boards_1056 = len(explore(GameConfiguration(10, 5, 6)))

print(f"There are {num_boards_636} possible configurations of Nannon{{6, 3, 6}}.")
print(f"There are {num_boards_846} possible configurations of Nannon{{8, 4, 6}}.")
print(f"There are {num_boards_1056} possible configurations of Nannon{{10, 5, 6}}.")

There are 2529 possible configurations of Nannon{6, 3, 6}.
There are 31102 possible configurations of Nannon{8, 4, 6}.
There are 367241 possible configurations of Nannon{10, 5, 6}.


### ~~Part 3: Tournaments~~

_This section involves a lot of output, so I've put it at the end so that you don't have to scroll through it all. I'll skip it for the time being._

### Part 4: Knowledge-Based Player

This was already addressed in detail above (including with head-to-head results in the table), but I'll show a tournament here too:

In [6]:
# There's a slightly different function for head-to-head tournaments for UI reasons
from play_tournament import play_tournament

win_ratio = play_tournament(FirstPlayerAlgorithm(), KnowledgePlayerAlgorithm(), rounds=5000, seed=98765, config=GameConfiguration(6, 3, 6))

Playing a Nannon{6,3,6} tournament of 5000 games, first vs knowledge. seed = 98765
Played 5000 games first vs knowledge: first won 1459 (29.18%). Took 5s.                                                        


### Part 3: Tournaments

For ease of grading, I'll show two games now. That's a lot of output, so I've put it here at the end. You can also see and/or play these games interactively through the CLI.

In [7]:
from play_game import play_game

# Let's play just one game of Nannon{6,3,6}
winner6 = play_game(
    FirstPlayerAlgorithm(), LastPlayerAlgorithm(),
    seed=12345, config=GameConfiguration(6, 3, 6))

# It works for higher-order Nannon too, like Nannon{12,5,6}
winner12 = play_game(
    FirstPlayerAlgorithm(), LastPlayerAlgorithm(),
    seed=56789, config=GameConfiguration(12, 5, 6))

Playing a Game of Nannon{6,3,6}! Black: first, White: last. Seed = 12345
White (last) rolled 2 and played:
           ● Black →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ●  ●     ○  ○  -  |
Home → ●   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ○ ← Home
                        ← White ○
Black (first) rolled 1 and played:
           ● Black →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ●  -  ●  ○  ○     |
Home → ●   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ○ ← Home
                        ← White ○
White (last) rolled 3 and played:
           ● Black →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ●  ○  ●  ○  -     |
Home → ●   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ○ ← Home
                        ← White ○
Black (first) rolled 3 and played:
           ● Black →
Goal →     |  ▼  ▼  ▼  ▼  ▼  ▼  |     ← Goal
           |  ●  ○  -  ○     ●  |
Home → ●   |  ▲  ▲  ▲  ▲  ▲  ▲  |   ○ ← Home
                        ← White ○
White (last) rolled 2 and played:
           ● Black →
Goal →     |  ▼  ▼